# Pothole detection

## Import libraries

In [ ]:
import keras
import numpy as np
import keras.preprocessing.image as img
import os

from keras.applications import ResNet50
from keras.layers import Dense
from keras.layers.pooling import GlobalMaxPool2D
from keras.models import Model
from keras.optimizers import SGD, Adam

from keras import backend as K

import matplotlib
import matplotlib.pyplot as plt

import csv

from PIL import Image

%matplotlib inline

## Import annotations

This part is not necessary when using `.flow_from_directory()`.

In [ ]:
with open('labels_bin.txt', newline='') as inputfile:
    train_ann = list(csv.reader(inputfile))
    
train_ann = train_ann[1:]
    
#with open('data_complex/complexTestFullSizeAllPotholes.txt', newline='') as inputfile:
#    test_ann = list(csv.reader(inputfile))

In [ ]:
train_files = [x[0].split(' ')[0].split('data/train/')[1] for x in train_ann]
train_bin = [int(x[0].split(' ')[1]) for x in train_ann]

## Create train and validation folders

In [ ]:
train_files = []
for path, subdirs, files in os.walk('data_crop/train/'):
    for name in files:
        train_files.append(os.path.join(path, name))

In [ ]:
np.random.shuffle(train_files)

valid_files = train_files[:500]
train_files = train_files[500:]

for f in valid_files:
    os.rename(f, 'data_crop/valid/' + f.split('data_crop/train/')[1])

## Crop images and save in a new folder

In [ ]:
all_files = []
for path, subdirs, files in os.walk('data'):
    for name in files:
        all_files.append(os.path.join(path, name))

In [ ]:
for f in all_files:
    temp_img = Image.open(f)
    temp_img = temp_img.crop((0, 600-435, 800, 600-435+185))
    temp_img.save('data_crop' + f.split('data')[1])

## Setup data generators

In [2]:
def imagenet_mean(x):
    x = x[..., ::-1]
    x[..., 0] -= 103.939
    x[..., 1] -= 116.779
    x[..., 2] -= 123.68
    return x

In [3]:
train_gen = img.ImageDataGenerator(
    horizontal_flip=True,
    width_shift_range=0.05,
    height_shift_range=0.05,
    preprocessing_function=imagenet_mean
)
test_gen = img.ImageDataGenerator(
    preprocessing_function=imagenet_mean
)

In [4]:
batch_size=64
img_size = (300,300)

In [ ]:
train_batches = train_gen.flow_from_directory(
    'data_crop/train/',
    batch_size=batch_size,
    target_size = img_size
)

valid_batches = test_gen.flow_from_directory(
    'data_crop/valid/',
    batch_size=batch_size,
    target_size = img_size,
    shuffle=False
)

## Start Modelling

### ResNet50

In [6]:
base_model = ResNet50(include_top=False, input_shape=img_size + (3,))

In [ ]:
base_model.summary()

Add new classification head

In [7]:
ft_map = base_model.get_layer(index=-2).output

x = GlobalMaxPool2D()(ft_map)
x = Dense(2, activation = 'softmax')(x)

model = Model(base_model.input, x)

In [ ]:
model.summary()

First tune only the new classification layer

In [8]:
for layer in base_model.layers:
    layer.trainable = False

In [11]:
opt = Adam(0.001)#, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.fit_generator(train_batches, 
                    steps_per_epoch=np.ceil(train_batches.samples/batch_size), 
                    epochs=5, verbose=1, 
                    validation_data=valid_batches, 
                    validation_steps=np.ceil(valid_batches.samples/batch_size),
                    )

Epoch 1/10
56/56 [==============================] - 99s - loss: 2.8914 - acc: 0.6084 - val_loss: 3.9995 - val_acc: 0.6300
Epoch 2/10
56/56 [==============================] - 85s - loss: 1.5326 - acc: 0.7152 - val_loss: 2.0807 - val_acc: 0.7320
Epoch 3/10
56/56 [==============================] - 87s - loss: 1.2646 - acc: 0.7671 - val_loss: 1.1093 - val_acc: 0.7880
Epoch 4/10
56/56 [==============================] - 85s - loss: 1.0260 - acc: 0.8081 - val_loss: 1.3700 - val_acc: 0.7940
Epoch 5/10
56/56 [==============================] - 86s - loss: 0.9894 - acc: 0.8071 - val_loss: 1.5298 - val_acc: 0.7640
Epoch 6/10
 1/56 [..............................] - ETA: 66s - loss: 0.9974 - acc: 0.8125

KeyboardInterrupt: 

In [14]:
model.save_weights('models/rn50_cls_300300.h5')

In [15]:
K.set_value(model.optimizer.lr, 0.0001)

In [16]:
model.fit_generator(train_batches, 
                    steps_per_epoch=np.ceil(train_batches.samples/batch_size), 
                    epochs=3, verbose=1, 
                    validation_data=valid_batches, 
                    validation_steps=np.ceil(valid_batches.samples/batch_size),
                    )

Epoch 1/5
56/56 [==============================] - 97s - loss: 0.7658 - acc: 0.8395 - val_loss: 0.6093 - val_acc: 0.8580
Epoch 2/5
56/56 [==============================] - 86s - loss: 0.7065 - acc: 0.8445 - val_loss: 0.6242 - val_acc: 0.8500
Epoch 3/5
56/56 [==============================] - 85s - loss: 0.6702 - acc: 0.8539 - val_loss: 0.6066 - val_acc: 0.8560
Epoch 4/5
56/56 [==============================] - 85s - loss: 0.7114 - acc: 0.8440 - val_loss: 0.6608 - val_acc: 0.8540
Epoch 5/5
 2/56 [>.............................] - ETA: 65s - loss: 0.5978 - acc: 0.8516

KeyboardInterrupt: 

In [17]:
model.save_weights('models/rn50_cls_300300.h5')

In [18]:
for i,layer in enumerate(model.layers):
    print(i, layer.name)

0 input_1
1 conv1
2 bn_conv1
3 activation_1
4 max_pooling2d_1
5 res2a_branch2a
6 bn2a_branch2a
7 activation_2
8 res2a_branch2b
9 bn2a_branch2b
10 activation_3
11 res2a_branch2c
12 res2a_branch1
13 bn2a_branch2c
14 bn2a_branch1
15 add_1
16 activation_4
17 res2b_branch2a
18 bn2b_branch2a
19 activation_5
20 res2b_branch2b
21 bn2b_branch2b
22 activation_6
23 res2b_branch2c
24 bn2b_branch2c
25 add_2
26 activation_7
27 res2c_branch2a
28 bn2c_branch2a
29 activation_8
30 res2c_branch2b
31 bn2c_branch2b
32 activation_9
33 res2c_branch2c
34 bn2c_branch2c
35 add_3
36 activation_10
37 res3a_branch2a
38 bn3a_branch2a
39 activation_11
40 res3a_branch2b
41 bn3a_branch2b
42 activation_12
43 res3a_branch2c
44 res3a_branch1
45 bn3a_branch2c
46 bn3a_branch1
47 add_4
48 activation_13
49 res3b_branch2a
50 bn3b_branch2a
51 activation_14
52 res3b_branch2b
53 bn3b_branch2b
54 activation_15
55 res3b_branch2c
56 bn3b_branch2c
57 add_5
58 activation_16
59 res3c_branch2a
60 bn3c_branch2a
61 activation_17
62 res3c

Fine-tune deeper layers - either conv5 block or conv5 + conv4

In [19]:
for layer in model.layers[:141]:
    layer.trainable = False
    
for layer in model.layers[141:]:
    layer.trainable = True

In [20]:
#opt = SGD(0.0001, momentum=0.9)
opt = Adam(0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
model.fit_generator(train_batches, 
                    steps_per_epoch=np.ceil(train_batches.samples/batch_size), 
                    epochs=5, verbose=1, 
                    validation_data=valid_batches, 
                    validation_steps=np.ceil(valid_batches.samples/batch_size),
                    )

Epoch 1/10
56/56 [==============================] - 102s - loss: 0.4623 - acc: 0.9059 - val_loss: 0.2567 - val_acc: 0.9300
Epoch 2/10
56/56 [==============================] - 94s - loss: 0.1467 - acc: 0.9654 - val_loss: 0.1766 - val_acc: 0.9640
Epoch 3/10
56/56 [==============================] - 94s - loss: 0.0984 - acc: 0.9788 - val_loss: 0.1815 - val_acc: 0.9660
Epoch 4/10
 2/56 [>.............................] - ETA: 81s - loss: 0.1058 - acc: 0.9844

KeyboardInterrupt: 

In [22]:
model.save_weights('models/rn50_crop_block5_300300.h5')

In [23]:
K.set_value(model.optimizer.lr, 0.00001)

In [24]:
model.fit_generator(train_batches, 
                    steps_per_epoch=np.ceil(train_batches.samples/batch_size), 
                    epochs=3, verbose=1, 
                    validation_data=valid_batches, 
                    validation_steps=np.ceil(valid_batches.samples/batch_size),
                    )

Epoch 1/3
56/56 [==============================] - 103s - loss: 0.0630 - acc: 0.9841 - val_loss: 0.1751 - val_acc: 0.9680
Epoch 2/3
56/56 [==============================] - 94s - loss: 0.0650 - acc: 0.9865 - val_loss: 0.1637 - val_acc: 0.9720
Epoch 3/3
56/56 [==============================] - 94s - loss: 0.0397 - acc: 0.9891 - val_loss: 0.1195 - val_acc: 0.9800


In [25]:
model.save_weights('models/rn50_crop_block5_300300.h5')

In [ ]:
K.set_value(model.optimizer.lr, 0.0000001)

In [ ]:
model.fit_generator(train_batches, 
                    steps_per_epoch=np.ceil(train_batches.samples/batch_size), 
                    epochs=2, verbose=1, 
                    validation_data=valid_batches, 
                    validation_steps=np.ceil(valid_batches.samples/batch_size),
                    )

### Check localisations

In [ ]:
pool_extract = Model(base_model.input, model.get_layer(index=-3).output)

In [ ]:
temp_img = img.load_img('data_crop/valid/positive/admvBpZvBLhveEM.JPG')
temp_img

In [ ]:
temp_arr = np.expand_dims(img.img_to_array(temp_img.resize((400,300))), 0)

In [ ]:
plt.imshow(temp_arr[0].astype('uint8'))

In [ ]:
#cmap = (np.dot(pool_extract.predict(temp_arr)[0], W) + b)
cmap = np.mean(pool_extract.predict(temp_arr)[0], axis=-1)

In [ ]:
plt.imshow(cmap)

### Evaluate on hold-out set

In [ ]:
model.predict_generator(valid_batches, steps=np.ceil(valid_batches.samples/batch_size), verbose=1)

In [57]:
valid_batches.reset()
x_valid = np.vstack([valid_batches.next()[0] for x in range(int(np.ceil(valid_batches.samples/batch_size)))])

In [58]:
valid_batches.reset()
y_valid = np.vstack([valid_batches.next()[1] for x in range(int(np.ceil(valid_batches.samples/batch_size)))])

In [70]:
p_valid = np.zeros_like(y_valid)
for flip in [False, True]:
    temp_x = x_valid
    if flip:
        temp_x = img.flip_axis(temp_x, axis=2)
    p_valid += 0.5 * model.predict(temp_x, verbose=1)

500/500 [==============================] - 11s    


In [71]:
np.mean(np.argmax(p_valid, axis=1) == np.argmax(y_valid, axis=1), axis=0)

0.97799999999999998